# Scarico gli id dei replay

In [21]:
import json
replay = []
i=0
with open('gen5_random_battle_replays.jsonl', 'r') as f:
    for lines in f:
        l = json.loads(lines)
        replay.append(l['id'].split('-')[-1])

# Scarico i relativi log

In [148]:
import requests
for id in replay:
    response = requests.get(f'https://replay.pokemonshowdown.com/gen5randombattle-{id}.json')
    if response.status_code != 200: continue
    battle = response.json()
    players = battle['players']
    battle = battle['log'].replace(players[0], 'p1a').replace(players[1], 'p2a').split('\n')
    battle = battle[battle.index('|start'):] # taglio via i log precedenti a  di |start
    team = {}
    # Estraggo i pokemon e le mosse
    for row in battle:
        if '|switch|p1a:' in row:
            pkm = row[12:].strip().split('|')[1] 
            pkm = pkm.split(', ')
            if pkm[0] not in team.keys():
                team[pkm[0]]={'liv': pkm[1], 'mosse': [], 'abilità': ''}
        if '|move|p1a:' in row:
            line = row[11:].split('|')
            pkm, move = line[0], line[1]
            if team.get(pkm) and move not in team.get(pkm)['mosse']:
                team.get(pkm)['mosse'].append(move)
        if 'ability|p1a:' in row:
            line = row[12:].split('|')
            pkm, ability = line[0], line[1]
            team.get(pkm)['abilità']=ability

    # Divido in turni:
    turni, actions= {}, {}
    nturno = 0
    for index, row in enumerate(battle):
        # Ogni volta che scatta un nuovo turno vedo chi c'è in campo e aggiorno il turno corrente
        if '|turn|' in row:
            nturno = int(row.strip('|turn|').strip()) # indica il turno successivo: dove dovrò scaricare le mosse
            current_turn = battle[:index]
            # # vedo che pokemon ci sono in campo
            pkm, pkm_avv = False, False
            for row0 in current_turn[::-1]:
                if '|switch|p1a: ' in row0:
                    pkm = row0[13:].split('|')[0]
                if '|switch|p2a: ' in row0:
                    pkm_avv = row0[13:].split('|')[0]
                if pkm and pkm_avv: break
            turni[nturno-1] = {'pkm': pkm, 
                               'pkm avversario': pkm_avv, 
                               'morti': [r[12:] for r in current_turn if '|faint|p1a: ' in r],
                               'log': '\n'.join(current_turn)}
        if nturno > 0:
            if '|move|p1a: ' in row:
                move = row[11:].split('|')[1]
                actions[nturno] = f'move|{move}'
                continue
            if '|switch|p1a: ' in row:
                pkm = row[13:].split('|')[0]
                actions[nturno] = f'switch|{pkm}'
                continue

    # Template del Prompt
    prompt="""I am a Pokémon trainer starting the turn {}.
You've this pokémon in the field: {}.
Your opponent have this pokémon: {}.
You have these switches available:
{}."""
    
    # Creo dataset
    dataset =  []
    for t in range(1,  nturno-1): # nturn contiene l'ultimo turno
        dataset.append(
            [
                {'role': 'system',
                 'content': prompt.format(*[
                    t, # turno successivo
                    str(turni[t-1]['pkm']+' '+str(team[turni[t-1]['pkm']])), # pokemon mio
                    turni[t-1]['pkm avversario'], # pokemon avversario
                    '\n'.join([f"- {p} | {info}{' | fainted' if p in turni[t-1]['morti'] else ''}" for p, info in team.items() if p!=turni[t-1]['pkm']]) # Cambi disponibili
                    ])
                    },
                {'role': 'user', 'content': turni[t-1]['log']},
                {'role': 'assistant', 'content': actions.get(t)}
                ]
        )